In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import itertools

import pandas as pd

model = ConcreteModel()

def fsmodel():
    model = ConcreteModel()
    
    # 5D Array
    model.gen = Set(initialize=[1, 2])  # Generators
    model.Periods = Set(initialize=[1, 2, 3])  # Time periods
    model.days = Set(initialize=[1, 2, 3])  # Demand scenario for time period 1
    
    # Total Scenarios
    
    file_path = 'newgenexp.xlsx'
    sheet_name = 'DIC'  # Specify the sheet name or sheet number
    
    # Load the data into a pandas DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    c = {}
    for i in range(0,df.shape[0]):
        c[df['Generator'][i]] = df['Ammortized Fixed operating cost($)'][i]
    model.c = Param(model.gen, initialize=c)  # Fixed cost per unit capacity of generator j
    
    sheet_name = 'FIXOP'  # Specify the sheet name or sheet number
    
    # Load the data into a pandas DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    f = {}
    for i in range(0,df.shape[0]):
        f[(df['Generator'][i],df['Period'][i])] = df['FIXOP'][i]
    model.f = Param(model.gen,model.Periods, initialize=f)  # Fixed cost per unit capacity of generator j
    
    sheet_name = 'Additional cost'  # Specify the sheet name or sheet number
    
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    g = {}
    for i in range(0,df.shape[0]):
        g[df['Day'][i]] = df['Cost'][i]
    model.g = Param(model.days, initialize=g)
    
    
    
    d = {}
    
    sheet_name = 'Demand'
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    for i in range(0,df.shape[0]):
        d[df['Period'][i],df['Day'][i]]= df['Demand (KWh)'][i]
        
                           
                   
    model.d = Param(model.Periods,model.days, initialize=d)  # Fixed cost per unit capacity of generator j
    
                           
       
    a = {}    
    sheet_name = 'Availability'
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    for i in range(0,df.shape[0]):
        a[df['Period'][i],df['Day'][i],df['Generator'][i]]= df['Availibility'][i]
        
                           
    model.a = Param(model.Periods,model.days,model.gen, initialize=a)  # Fixed cost per unit capacity of generator j
    
    # Variables
    model.x = Var(model.gen, domain=NonNegativeReals)  # Installed capacity of generator j
    model.y = Var(model.Periods, model.days, model.gen, domain=NonNegativeReals)  # Operating level of generator j
    model.y_purchased = Var(model.Periods, model.days, domain=NonNegativeReals)  # Additional capacity purchased
    # Objective function
    def objective_rule(model):
        fixed_cost = 1*sum(model.c[j] * model.x[j] for j in model.gen)
        operating_cost = sum(sum(sum(model.f[j,i] * model.y[i,k, j] for j in model.gen) + model.g[k] * model.y_purchased[i,k] for i in model.Periods) for k in model.days)
        return fixed_cost + operating_cost
    
    model.obj = Objective(rule=objective_rule, sense=minimize)
    
    # Constraints
    #Constraint for fixing x values
    

    # Demand satisfaction constraints
    def demand_satisfaction_rule(model, i, k):
        return sum(model.y[i,k,j] for j in model.gen) + model.y_purchased[i,k] >= model.d[i,k]
    model.demand_satisfaction_constraint = Constraint(model.Periods, model.days, rule=demand_satisfaction_rule)
    
    # Availability constraints
    def availability_rule(model, i, k, gen):
        return model.y[i,k,gen] <= model.a[i,k,gen] * model.x[gen]
    model.availability_constraint = Constraint(model.Periods, model.days, model.gen, rule=availability_rule)
    return model


In [2]:
def hlmodel(param_val):
    model = ConcreteModel()
    
    # 5D Array
    model.gen = Set(initialize=[1, 2])  # Generators
    model.Periods = Set(initialize=[1, 2, 3])  # Time periods
    model.days = Set(initialize=[1, 2, 3])  # Demand scenario for time period 1
    
    # Total Scenarios
    
    file_path = 'newgenexp.xlsx'
    sheet_name = 'DIC'  # Specify the sheet name or sheet number
    
    # Load the data into a pandas DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    c = {}
    for i in range(0,df.shape[0]):
        c[df['Generator'][i]] = df['Ammortized Fixed operating cost($)'][i]
    model.c = Param(model.gen, initialize=c)  # Fixed cost per unit capacity of generator j
    
    sheet_name = 'FIXOP'  # Specify the sheet name or sheet number
    
    # Load the data into a pandas DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    f = {}
    for i in range(0,df.shape[0]):
        f[(df['Generator'][i],df['Period'][i])] = df['FIXOP'][i]
    model.f = Param(model.gen,model.Periods, initialize=f)  # Fixed cost per unit capacity of generator j
    
    sheet_name = 'Additional cost'  # Specify the sheet name or sheet number
    
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    g = {}
    for i in range(0,df.shape[0]):
        g[df['Day'][i]] = df['Cost'][i]
    model.g = Param(model.days, initialize=g)
    
    
    
    d = {}
    
    sheet_name = 'Demand'
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    for i in range(0,df.shape[0]):
        d[df['Period'][i],df['Day'][i]]= df['Demand (KWh)'][i]
        
                           
                   
    model.d = Param(model.Periods,model.days, initialize=d)  # Fixed cost per unit capacity of generator j
    
                           
       
    a = {}    
    sheet_name = 'Availability'
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    for i in range(0,df.shape[0]):
        a[df['Period'][i],df['Day'][i],df['Generator'][i]]= df['Availibility'][i]
        
                           
    model.a = Param(model.Periods,model.days,model.gen, initialize=a)  # Fixed cost per unit capacity of generator j
    
    # Variables
    model.x = Var(model.gen, domain=NonNegativeReals)  # Installed capacity of generator j
    model.y = Var(model.gen, domain=NonNegativeReals)  # Operating level of generator j
    model.y_purchased = Var(domain=NonNegativeReals)  # Additional capacity purchased
    # Objective function
    def objective_rule(model):
        fixed_cost = 1*sum(model.c[j] * model.x[j]*1 for j in model.gen)
        operating_cost = sum(sum(sum(model.f[j,i] * model.y[j] for j in model.gen) + model.g[k] * model.y_purchased for i in model.Periods) for k in model.days)
        return fixed_cost + operating_cost
    
    model.obj = Objective(rule=objective_rule, sense=minimize)
    
    

    # Demand satisfaction constraints
    def demand_satisfaction_rule(model):
        return sum(model.y[j] for j in model.gen) + model.y_purchased >= sum(model.d[i,k] for i in model.Periods for k in model.days)
    model.demand_satisfaction_constraint = Constraint(rule=demand_satisfaction_rule)
    
    # Availability constraints
    def availability_rule(model, gen):
        return model.y[gen] <= sum((model.a[i,k,gen] for i in model.Periods for k in model.days)) * model.x[gen]*param_val[gen-1]
    model.availability_constraint = Constraint(model.gen, rule=availability_rule)

    def min_cap_rule(model, gen):
        return model.y[gen] >= param_val[2]
    model.mincap_constraint = Constraint(model.gen, rule=min_cap_rule)
    opt = SolverFactory('gurobi')
    results = opt.solve(model, tee=True)
    
    
    #model.display()
    if results.solver.termination_condition == TerminationCondition.infeasible:
        return 10**20
    xval = {1:model.x[1](),2:model.x[2]()}
    return xval
    

In [3]:
def llmodel(xval):
    if(xval==10**20):
        return 10**20
    model = fsmodel()
    def cap_fix_rule(model, gen):
        return model.x[gen] == xval[gen]
    model.cap_fix_constraint = Constraint(model.gen, rule=cap_fix_rule)
    opt = SolverFactory('gurobi')
    results = opt.solve(model, tee=True)
    
    
    #model.display()
    if results.solver.termination_condition == TerminationCondition.infeasible:
        return 10**20
    return value(model.obj)

In [4]:
def MBBF(param):
    xval = hlmodel(param)
    print(xval)
    obj = llmodel(xval)
    return obj


In [ ]:
from mads import mads
design_variables = [1,1,0]
bounds_lower = [0,0,0]
bounds_upper = [1,1,2100]

dp_tol = 1E-6                               # Minimum poll size stopping criteria
nitermax = 2000                             # Maximum objective function evaluations
dp = 0.5                                    # Initial poll size as percent of bounds
dm = 0.5                                   # Initial mesh size as percent of bounds
mads.orthomads(design_variables, bounds_upper, bounds_lower, MBBF, dp, dm, dp_tol, nitermax, True, True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file C:\Users\aramanuj\AppData\Local\Temp\tmp6xjtx7zi.pyomo.lp
Reading time = 0.00 seconds
x1: 5 rows, 5 columns, 9 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 9 nonzeros
Model fingerprint: 0x099e300a
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+03, 9e+03]
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5683498e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0